In [ ]:
import numpy as np
import pandas as pd
import numba as nb
import matplotlib.pyplot as plt
import time
from pathlib import Path
from IDT_alg_VR_centred_Varjo import IDTVR

In [ ]:
path = Path("/run/media/weiss/Elements/cl_decisions_data/3uy2Zr/2022-11-07-10h32m-CognitiveLoad-raw-et-data.csv")
df_et = pd.read_csv(
    path,
    sep=";",
    encoding="utf-8",
)

In [ ]:
year = path.name.split("-")[0]
month = path.name.split("-")[1]
day = path.name.split("-")[2]
print(year, month, day)

In [ ]:
# fix LogTime
df_et["LogDate"] = year + "-" + month + "-" + day + " " + df_et["LogTime"]
df_et["LogDate"] = pd.to_datetime(df_et["LogDate"], format="%Y-%m-%d %H.%M.%S.%f")

In [ ]:
# head position
df_et["HMDPosition"] = df_et["HMDPosition"].str.strip("()")
df_et[["head_pos_x", "head_pos_y", "head_pos_z"]] = df_et[
    "HMDPosition"
].str.split(",", expand=True).astype(float)

In [ ]:
# gaze vector
df_et["CombinedGazeForward"] = df_et["CombinedGazeForward"].str.strip("()")
df_et[["et_x", "et_y", "et_z"]] = df_et["CombinedGazeForward"].str.split(
    ",", expand=True
).astype(float)

In [ ]:
df_et

In [ ]:
# convert timestamp to timedelta in seconds
firstTimestamp = df_et.loc[0, "LogDate"]
df_et["elapsedTime"] = df_et["LogDate"].apply(lambda x: (x - firstTimestamp).total_seconds())

In [ ]:
# calculate time_delta between two consecutive samples
df_et["time_delta"] = df_et["elapsedTime"] - df_et["elapsedTime"].shift(1)

In [ ]:
df = df_et[
    [
        "elapsedTime",
        "time_delta",
        "head_pos_x",
        "head_pos_y",
        "head_pos_z",
        "et_x",
        "et_y",
        "et_z",
        "RightIrisDiameterInMM",
        "LeftIrisDiameterInMM",
        "RightPupilIrisDiameterRatio",
        "LeftPupilIrisDiameterRatio",
        "FocusItem"
    ]
].copy()

# convert pupil values to float
for col in [ "RightIrisDiameterInMM",
        "LeftIrisDiameterInMM",
        "RightPupilIrisDiameterRatio",
        "LeftPupilIrisDiameterRatio",
]:
    df[col] = df[col].str.replace(",", ".")
    df[col] = df[col].astype(float)

print("Shape", df.shape)
print("Columns", df.columns)
df.head(5)


In [ ]:
idt_vr = IDTVR(numba_allow=True, time_th=0.1)

t0 = time.time()
df_et_res, df_et_fixations = idt_vr.fit_compute(df, time="elapsedTime", debug=True)
delta_time_numba = time.time() - t0

print("\n\nfix. algo duration: ", delta_time_numba)
df_et_fixations

In [ ]:
print(
    "Percentage of fixations detected",
    np.round(100 * np.mean(df_et_res["class_disp"] == 0), 2),
    "%",
)
print(
    "Percentage of saccades detected",
    np.round(100 * np.mean(df_et_res["class_disp"] == 1), 2),
    "%",
)
print("Delta time", np.round(delta_time_numba, 4), "s")


In [ ]:
df_et_fixations['duration'].describe()

In [ ]:
ivt_file = Path("C://Users/tbswe/data/et-data/VR/Logfiles_All/GazeData/p001_fixations.csv")

#fix enconding
fi = ivt_file.open('rb')
content = fi.read()
fi.close()
ivt_file_fixed = Path("C://Users/tbswe/data/et-data/VR/Logfiles_All/GazeData/p001_fixations_fixed.csv")
fo = ivt_file_fixed.open('wb')
fo.write(content.replace(b'\x00', b''))
fo.close()

df_ivt = pd.read_csv(
    ivt_file_fixed,
    sep=",",
    encoding="iso-8859-1",
)

In [ ]:
df_ivt['gazeevent_duration'].describe()